In [33]:
import pandas as pd
import anndata as ad
import os
import ntpath

In [14]:
data_dir = "../Training_Data"
patients = [os.path.basename(f) for f in os.scandir(data_dir) if f.is_dir()]
print(patients)
gene = "RUBCNL"
columns_of_interest = ["tile", gene]

In [44]:
# generate training dataframe with all training samples
for i in patients:
    print(i)
    adata = ad.read_h5ad(data_dir + "/" + i + "/Preprocessed_STDataset/tmm_combat_scaled_" + i + ".h5ad")
    st_dataset = adata.to_df()
    st_dataset["tile"] = st_dataset.index
    print(st_dataset.tile.iloc[0][0:-2])
    file_names = st_dataset.tile.iloc[0][0:-2]
    base_name = ntpath.basename(file_names)
    st_dataset['tile'] = st_dataset['tile'].apply(
        lambda x: ntpath.basename(x[0:-2]))
    filename = 
    st_dataset.to_csv(filename, index=False)
    break

In [46]:
st_dataset.head()

In [ ]:
#Preprocess spatial matrix and true-prediction dataset
#------------------------------------------------------------------------------
#Load spatial matrix
def spatial_plot(data_dir, patient, test_df, true, preds, batch_size, lr, sample): 
  spatial_matrix = pd.read_csv(data_dir+patient+"/Preprocessed_STDataset/Raw_Spatial_Matrix_156_"+sample+".csv") #adapt to sample
  spatial_matrix['tile'] = spatial_matrix['tile'].apply(lambda x: "{}{}".format(x, ".tiff"))
  spatial_matrix['tile'] = spatial_matrix['tile'].apply(lambda x: "{}{}".format(data_dir+patient+"/Tiles_156/", x[57:])) #adapt to sample

#Merge spatial with test dataset to get spatial coordinates for predicted tiles
  merge = pd.merge(test_df, spatial_matrix, on = "tile")

#Get filtered spatial matrix
  new_spatial = merge.copy()
  new_spatial = new_spatial[["tile", "x", "y"]]
  new_spatial.set_index("tile", inplace=True)
  spatial_array = new_spatial.to_numpy()

#Get filtered gene matrix
  new_gene = merge.copy()
  new_gene.drop(columns=["x", "y"], inplace=True)
  new_gene.set_index("tile", inplace=True)

#Load Image
  Image.MAX_IMAGE_PIXELS = None
  h_e_img = plt.imread(data_dir+patient+"/Preprocessed_STDataset/HE_norm_"+patient+".tif") #adapt to sample

#Setup anndata object
  adata_test = AnnData(new_gene, obsm={"spatial": spatial_array})
  adata_test.var
  spatial_key = "spatial"
  library_id = "tissue42"
  adata_test.uns[spatial_key] = {library_id: {}}
  adata_test.uns[spatial_key][library_id]["images"] = {}
  adata_test.uns[spatial_key][library_id]["images"] = {"hires": h_e_img}
  adata_test.uns[spatial_key][library_id]["scalefactors"] = {"tissue_hires_scalef": 1, "spot_diameter_fullres": 130}

#Save predicted anndata object
  adata_test.write(data_dir + "ST_Predict_absolute_multi_"+ pretrained_model + "_" + str(batch_size) +"_" + lr +"_" +load_metric+"_"+sample+".h5ad") #adapt to sample

#Plot spatial gene truth and prediction
  for l in range(len(true)):
    plot_name = data_dir+"/Figures/ST_Predict_absolute_multi_"+true[l].replace("_true", "")+ "_" + str(batch_size)+"_"+lr+"_"+pretrained_model+"_valid_spatial_"+load_metric+"_"+sample+".tif" #adapt to sample
    sk={"fixed_value": 500, "location": "lower left", "box_alpha": 0.5}
    color=[true[l], preds[l]]
    title=[true[l].replace("_true", "")+" - groundtruth", preds[l].replace("_prediction", "")+" - prediction"]
    sq.pl.spatial_scatter(adata_test, use_raw=False, shape="circle", color=color, title=title, alpha = 0.8, cmap="bwr", axis_label=["", ""], scalebar_dx=0.72, scalebar_units="um", scalebar_kwargs=sk, save = plot_name)
#------------------------------------------------------------------------------


pearson_dict = {}

with open(test_log, "a") as f:
  f.write("Batchsize: " + str(batch_size)) #adapt to batch size
  f.write("\n")
  f.write("Learning Rate: " + lr) #adapt to learning rate
  f.write("\n")
  f.write("Optimizer: AdamW")
  f.write("\n")
  f.write("Exponential LR scheduler: 0.9")
  f.write("\n")
  f.write("Loss Function: MSE Loss")
  f.write("\n")
  f.write("Metric for parameter saved: " + load_metric)
  f.write("\n")
  f.write("Sample: " + sample)
  f.write("\n")
    
model = get_model(pretrained_model)
model_path = '../currentModel/model.pt'
model.load_state_dict(torch.load(model_path)) #adapt to file.pt and directory with best trained parameters of model
model.to(device)
    
test_loader = DataLoader(loaded_test_dataset, batch_size=batch_size, shuffle=False)
    
test_df = testing(gene_list, test_loader, model)
    
true = list(test_df)[1::2]
preds = list(test_df)[2::2]
    
pearson_calc(test_df, true, preds, pearson_dict, batch_size, lr, sample)
    
spatial_plot(test_df, true, preds, batch_size, lr, sample)
    
pearson_df = pd.DataFrame([pearson_dict])
pearson_df = pearson_df.T

pearson_df_save = data_dir + "ST_Predict_absolute_multi_"+ pretrained_model + "_" + str(batch_size) +"_" + lr +"_pearson_metrics_" +load_metric+"_"+sample+".csv" #adapt to sample
pearson_df.to_csv(pearson_df_save, header=False, index=True)

test_df_save = data_dir + "ST_Predict_absolute_multi_"+ pretrained_model + "_" + str(batch_size) +"_" + lr +"_df_" +load_metric+"_"+sample+".csv" #adapt to sample
test_df.to_csv(test_df_save, header=True, index=False)

with open(test_log, "a") as f:
  f.write("--------------------------------------------")
  f.write("\n")